In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#base imports 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

#display image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#deep learing 
import tensorflow as tf

#splitting the data
#splitting the data 
from sklearn.model_selection import train_test_split


In [ ]:
img_height=500
img_width=600
img = mpimg.imread("../input/cassava-leaf-disease-classification/train_images/1000015157.jpg")
imgplot = plt.imshow(img[:img_width,:img_height])
plt.show()
img.shape


In [ ]:
#reading json file 
import json

with open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json") as f:
  data = json.load(f)
data

In [ ]:
df= pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")

df

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential(
    [
     layers.experimental.preprocessing.RandomRotation(0.05),
     layers.experimental.preprocessing.RandomCrop(28, 28)
    ]
)

model = tf.keras.Sequential([
    data_augmentation,

    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(4096, activation='relu'),
    layers.Dense(10)
])

In [ ]:
import glob
image_path= glob.glob("../input/cassava-leaf-disease-classification/train_images/*.jpg")
labels= pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
labels["label"]
d = {'path': image_path, 'label': labels["label"]}
df = pd.DataFrame(data=d)
df=df.iloc[:5000]
labels.label

X_train, X_test, y_train, y_test = train_test_split(df.path, df.label, test_size=0.33, random_state=42)

X_train=X_train.tolist()
X_test=X_test.tolist()
y_train=y_train.tolist()
y_test=y_test.tolist()


In [ ]:
# directory="../input/cassava-leaf-disease-classification/train_images"
# data=tf.keras.preprocessing.image_dataset_from_directory(
#     directory,
#     label_mode="int",
#     class_names=None,
#     color_mode="rgb",
#     batch_size=32,
#     image_size=(256, 256),
#     shuffle=True,
#     seed=None,
#     validation_split=None,
#     subset=None,
#     interpolation="bilinear",
#     follow_links=False,
# )

In [ ]:
x_files="../input/cassava-leaf-disease-classification/train_images"
train_data = tf.data.Dataset.from_tensor_slices((X_train,y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_test,y_test))

def process_img(file_path,label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, size=(img_height, img_width))
    return (img,label)

train_data = train_data.map(process_img).batch(10)
test_data = test_data.map(process_img).batch(10)


img,target = next(iter(train_data))

In [ ]:
#predicting function 
from sklearn.metrics import confusion_matrix

''' the idea of this function to make the prediction accouding to the input.'''
def prediction_matrix(dataframe,model,size):
    #sorting the value wrt to the label 
    df=dataframe.sort_values(by=['label'])
    #picking the 10
    df=df.iloc[:size]
    #getting the image path
    x_train=df.path
    x_train=x_train.tolist()
    #getting the image label
    y_train=df.label
    y_train=X_train.tolist()
    #creating the dataset object
    train_data = tf.data.Dataset.from_tensor_slices((X_train,y_train))
    train_data = train_data.map(process_img).batch(size)
    
    #passing the matrix to model 
    predictions=model.predict(train_data)
    predicted_class_list=[]
    for x in predictions:
        score = tf.nn.softmax(x)
        predicted_class=np.argmax(score)
        predicted_class.append(predicted_class)
        
    return predicted_class_list
#     confusion_matrix(train_data, predicted_class_list)
    

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset=dataset.batch(5)
iterator = iter(dataset)
print(iterator.get_next())

print(iterator.get_next())


In [ ]:
import keras.layers.experimental
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten,experimental
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

model_vgg_lite = Sequential()
num_classes=5
# First CONV-ReLU Layer
model_vgg_lite.add(Conv2D(64, (3, 3), padding = 'same', input_shape=(img_height, img_width, 3)))
model_vgg_lite.add(Activation('relu'))
model_vgg_lite.add(BatchNormalization())

# Second CONV-ReLU Layer
model_vgg_lite.add(Conv2D(64, (3, 3), padding = "same", input_shape=(img_height, img_width, 3)))
model_vgg_lite.add(Activation('relu'))
model_vgg_lite.add(BatchNormalization())

# Max Pooling with Dropout 
model_vgg_lite.add(MaxPooling2D(pool_size=(2, 2)))
model_vgg_lite.add(Dropout(0.2))

# 3rd set of CONV-ReLU Layers
model_vgg_lite.add(Conv2D(128, (3, 3), padding="same"))
model_vgg_lite.add(Activation('relu'))
model_vgg_lite.add(BatchNormalization())

# 4th Set of CONV-ReLU Layers
model_vgg_lite.add(Conv2D(128, (3, 3), padding="same"))
model_vgg_lite.add(Activation('relu'))
model_vgg_lite.add(BatchNormalization())

# Max Pooling with Dropout 
model_vgg_lite.add(MaxPooling2D(pool_size=(2, 2)))
model_vgg_lite.add(Dropout(0.2))

# 5th Set of CONV-ReLU Layers
model_vgg_lite.add(Conv2D(256, (3, 3), padding="same"))
model_vgg_lite.add(Activation('relu'))
model_vgg_lite.add(BatchNormalization())

# 6th Set of CONV-ReLU Layers
model_vgg_lite.add(Conv2D(256, (3, 3), padding="same"))
model_vgg_lite.add(Activation('relu'))
model_vgg_lite.add(BatchNormalization())

# Max Pooling with Dropout 
model_vgg_lite.add(MaxPooling2D(pool_size=(2, 2)))
model_vgg_lite.add(Dropout(0.2))

# First set of FC or Dense Layers
model_vgg_lite.add(Flatten())
model_vgg_lite.add(Dense(256))
model_vgg_lite.add(Activation('relu'))
model_vgg_lite.add(BatchNormalization())
model_vgg_lite.add(Dropout(0.5))

# Second set of FC or Dense Layers
model_vgg_lite.add(Dense(256))
model_vgg_lite.add(Activation('relu'))
model_vgg_lite.add(BatchNormalization())
model_vgg_lite.add(Dropout(0.5))

# Final Dense Layer
model_vgg_lite.add(Dense(num_classes))
model_vgg_lite.add(Activation("softmax"))
model_vgg_lite.add(Dense(num_classes)) 
##units are 1 because we are using binary activation

In [ ]:
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint("keras.model", save_best_only=True, verbose=1)
callbacks_list = [model_checkpoint]
model_vgg_lite.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model_vgg_lite.summary()


In [ ]:
epochs=50
history = model_vgg_lite.fit(
  train_data,validation_data=test_data,
  epochs=epochs,callbacks=callbacks_list
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
img = mpimg.imread("../input/cassava-leaf-disease-classification/train_images/1000015157.jpg")
img_array=tf.convert_to_tensor(
    img,)
img_array = tf.expand_dims(img[:img_width,:img_height], 0) # Create a batch

predictions=model_vgg_lite.predict(img_array)
score = tf.nn.softmax(predictions[0])
# score

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(np.argmax(score), 100 * np.max(score))
)

In [ ]:
# train_data.take(5)
# list(train_data.take(1).as_numpy_iterator())

predictions=model_vgg_lite.predict(train_data.take(1))
# prediction =train_data.take(3).map(displayimage)
# predictions.shape
score = tf.nn.softmax(predictions[0])
predictions
    

In [ ]:
for x in train_data.unbatch().take(1).as_numpy_iterator():
    print(x)